In [1]:
import SimpleITK as sitk
import pandas as pd
import os
import numpy as np

In [2]:
#
# open structure properties file
#
csv_file = 'structure_properties.csv'
df = pd.read_csv(csv_file)
df.set_index('acronym',inplace=True)

In [3]:
#
# open annotation file
#
model_directory = "C:/Users/lydia/OneDrive/Work/DigitalAssets/_UPENN_fMOST/"
annotation_file = os.path.join( model_directory,  "25um_V3", "CCFTemplate25umLabels.nii.gz")

annotation = sitk.ReadImage( annotation_file )

In [4]:
def create_mask( structure, annotation, df, fg = 1 ) :
    
    slist = df.loc[structure]['substructures']
    slist = slist.split('/')
    slist = [ int(x) for x in slist ]
    
    arr = sitk.GetArrayFromImage( annotation )
    idx = np.isin(arr,slist)
    output = np.zeros(arr.shape,np.uint16)
    output[idx] = fg
    
    output_image = sitk.GetImageFromArray( output )
    output_image.CopyInformation( annotation )    
    return output_image

In [5]:
def overlay( base, image ) :
    
    barr = sitk.GetArrayFromImage( base )
    iarr = sitk.GetArrayFromImage( image )
    
    idx = np.where( iarr > 0 )
    barr[idx] = iarr[idx]
    
    output_image = sitk.GetImageFromArray( barr )
    output_image.CopyInformation( base )    
    return output_image

In [6]:
def set_foreground_value( image, fg ) :
    
    arr = sitk.GetArrayFromImage( image )
    idx = np.where( arr > 0 )
    arr[idx] = fg
    
    output_image = sitk.GetImageFromArray( arr )
    output_image.CopyInformation( image )    
    return output_image    

In [7]:
def interface( image1, image2, fg ) :

    mask1 = set_foreground_value( image1, fg )
    mask2 = set_foreground_value( image2, fg )
    
    mask1 = sitk.BinaryDilate( mask1, (1,1,1), sitk.sitkBall, 0, fg )
    mask2 = sitk.BinaryDilate( mask2, (1,1,1), sitk.sitkBall, 0, fg )
    
    arr1 = sitk.GetArrayFromImage( mask1 )
    arr2 = sitk.GetArrayFromImage( mask2 )
    
    idx = np.logical_and( arr1 == fg, arr2  == fg )
    output = np.zeros(arr1.shape,np.uint16)
    output[idx] = fg
 
    output_image = sitk.GetImageFromArray( output )
    output_image.CopyInformation( mask1 )    
    return output_image

In [8]:
def overlap( image1, image2, fg ) :
    
    mask1 = set_foreground_value( image1, fg )
    mask2 = set_foreground_value( image2, fg )
    
    arr1 = sitk.GetArrayFromImage( mask1 )
    arr2 = sitk.GetArrayFromImage( mask2 )
    
    idx = np.logical_and( arr1 == fg, arr2  == fg )
    output = np.zeros(arr1.shape,np.uint16)
    output[idx] = fg
 
    output_image = sitk.GetImageFromArray( output )
    output_image.CopyInformation( mask1 )    
    return output_image    

In [9]:
def connected_components( image, size_threshold = 1000, fg = 1 ) :
    
    components = sitk.ConnectedComponent( image, True )
    arr = sitk.GetArrayFromImage( components )
    values, counts = np.unique( arr, return_counts = True )
    
    # order these by size descending
    ind = np.argsort(counts)
    values = values[np.flip(ind)]
    counts = counts[np.flip(ind)]
    
    output_images = []
    
    for v,c in zip(values,counts) :
        
        # exclude zero label
        if v == 0 :
            continue
        
        # remove components with size below threshold
        if c < 1000 :
            continue
            
        # create an image for the label
        idx = np.where( arr == v )
        output = np.zeros( arr.shape, np.uint16 )
        output[idx] = fg
        
        img = sitk.GetImageFromArray( output )
        img.CopyInformation( image )
        output_images.append( img )
        
    return output_images
    

In [10]:
labels = {}
labels['mid sagittal'] = 1
labels['root'] = 2
labels['grey'] = 3
labels['fiber tracts'] = 4
labels['VS'] = 5
labels['fi'] = 6
labels['chpl'] = 7
labels['hbc'] = 8
labels['chpl posterior'] = 9
labels['chpl lateral'] = 10
labels['chpl anterior'] = 11

In [11]:
#
# -- midsagittal mask
#
mid_sagittal = sitk.Image(annotation.GetSize(), sitk.sitkUInt16)
mid_sagittal[:,:,:] = 0
mid_sagittal[227:229,:,:] = labels['mid sagittal']

In [12]:
#
# --- main classes: root/outside, grey matter, fiber tracts and ventricular system
#
root = create_mask('root', annotation, df, labels['root'] )
grey = create_mask('grey', annotation, df, labels['grey'] )
fiber_tracts = create_mask( 'fiber tracts', annotation, df, labels['fiber tracts'] )
VS = create_mask('VS', annotation, df, labels['VS'] )

In [13]:
# --- reclass the interface of MB and CB as root/outside
MB = create_mask('MB', annotation, df, labels['grey'] )
CB = create_mask('CB', annotation, df, labels['grey'] )
MB_CB_interface = interface( MB, CB, fg = labels['root'] )

In [14]:
# --- reclass the interface of PAG and CBX as ventricles
PAG = create_mask('PAG', annotation, df, labels['grey'] )
CBX = create_mask('CBX', annotation, df, labels['grey'] )
PAG_CBX_interface = interface( PAG, CBX, labels['VS'] )

In [15]:
# -- reclass the interface of P and CBX as ventricles
P = create_mask('P', annotation, df, labels['grey'] )
P_CBX_interface = interface( P, CBX, labels['VS'] )

In [16]:
# -- reclass the interface of MY and CBX as root/outside
MY = create_mask('MY', annotation, df, labels['grey'] )
MY_CBX_interface = interface( MY, CBX, labels['root'] )

In [17]:
# -- reclass the interface of CTXpl and MB as root/outside
CTXpl = create_mask('CTXpl', annotation, df, labels['grey'] )
MB_CTXpl_interface = interface( MB, CTXpl, labels['root'] )

In [18]:
# -- reclass the interface of TH and CTXpl as root/outside
TH = create_mask('TH', annotation, df, labels['grey'] )
TH_CTXpl_interface = interface( TH, CTXpl, labels['root'] )

In [19]:
# -- reclass PVpo on the mid-sagittal mask to VS
PVpo = create_mask('PVpo', annotation, df, labels['grey'])
PVpo_mid_sagittal_masked = overlap( PVpo, mid_sagittal, labels['VS'] )

In [20]:
# -- reclass OLF on the mid-sagittal mask to root/outside
OLF = create_mask('OLF', annotation, df, labels['grey'])
OLF_mid_sagittal_masked = overlap( OLF, mid_sagittal, labels['root'] )

In [21]:
# -- reclass the bsc as root/outside
bsc = create_mask('bsc', annotation, df, labels['fiber tracts'] )
bsc_as_root = set_foreground_value( bsc, labels['root'] )

In [22]:
# --make fimbra a distinct landmark
fi = create_mask('fi', annotation, df, labels['fi'] )

In [23]:
# -- make hbc a distinct landmark
hbc = create_mask('hbc', annotation, df, labels['hbc'] )

In [24]:
# - divide choriod plexeus into 3 distinct landmarks
# -- posterior (largest)
# -- lateral (reunify the component in each hemisphere)
# -- anterior (smallest above threshold)
chpl = create_mask('chpl', annotation, df, labels['chpl'] )

components = connected_components( chpl, 1000, labels['chpl'] )
chpl_posterior = set_foreground_value( components[0], labels['chpl posterior'] )
chpl_lateral = set_foreground_value( overlay( components[1], components[2] ), labels['chpl lateral'] )
chpl_anterior = set_foreground_value( components[3], labels['chpl anterior'] )

In [25]:
# -- combined the landmarks into single volume
combined = overlay( root, grey )
combined = overlay( combined, MB_CB_interface )
combined = overlay( combined, PAG_CBX_interface )
combined = overlay( combined, P_CBX_interface )
combined = overlay( combined, MY_CBX_interface )
combined = overlay( combined, MB_CTXpl_interface )
combined = overlay( combined, TH_CTXpl_interface )
combined = overlay( combined, PVpo_mid_sagittal_masked )
combined = overlay( combined, OLF_mid_sagittal_masked )
combined = overlay(combined,fiber_tracts)
combined = overlay( combined, bsc_as_root )
combined = overlay( combined, hbc )
combined = overlay( combined, fi )
combined = overlay(combined,VS)
combined = overlay( combined, chpl_posterior )
combined = overlay( combined, chpl_lateral )
combined = overlay( combined, chpl_anterior )

In [26]:
sitk.WriteImage( combined, 'combined.nii.gz', True)